# Grill and BBQ Data Cleaning

## Setup

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
# Visualization
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn')

## Read in Data

In [3]:
# Read in the data
df = pd.read_csv('unzipped_data/grill-and-bbq.csv', sep = '|')
print("Shape of DataFrame", df.shape)
df.head()

Shape of DataFrame (288456, 5)


,PAGE,URL,COUNT,AMOUNT,INGREDIENT
0,1,https://www.bigoven.com/recipe/chicken-breasts...,8,2 tablespoons,canola oil
1,1,https://www.bigoven.com/recipe/chicken-breasts...,8,1/2 large,onion
2,1,https://www.bigoven.com/recipe/chicken-breasts...,8,6,pineapple
3,1,https://www.bigoven.com/recipe/chicken-breasts...,8,,pepper
4,1,https://www.bigoven.com/recipe/chicken-breasts...,8,1 1/2 cup,tomatillo


## Clean Data: General

### Drop metadata

'PAGE' and 'COUNT' are not needed for the analysis.

In [4]:
df.drop(['PAGE', 'COUNT'], axis = 1, inplace = True)
df.head()

,URL,AMOUNT,INGREDIENT
0,https://www.bigoven.com/recipe/chicken-breasts...,2 tablespoons,canola oil
1,https://www.bigoven.com/recipe/chicken-breasts...,1/2 large,onion
2,https://www.bigoven.com/recipe/chicken-breasts...,6,pineapple
3,https://www.bigoven.com/recipe/chicken-breasts...,,pepper
4,https://www.bigoven.com/recipe/chicken-breasts...,1 1/2 cup,tomatillo


### Fix error in header

### Remove duplicates

In [5]:
print("shape of DataFrame before de-dup:", df.shape)
print("Number of duplicates:",sum(df.duplicated()))

shape of DataFrame before de-dup: (288456, 3)
Number of duplicates: 276769


In [6]:
# Remove duplicates
df = df.drop_duplicates()

print("shape of DataFrame after de-dup:", df.shape)

shape of DataFrame after de-dup: (11687, 3)


### Make all in lower case

In [7]:
for item in ['AMOUNT', 'INGREDIENT']: 
    df[item] = df[item].apply(lambda x: x.lower().strip())

## Look at 'URL' Column

### Create a column with dish id

In [8]:
# Write a function to get the dish id from URL
def get_dish_id(url): 
    dish_id = re.findall('\S*/([0-9]+)$', url)[0]
    return dish_id

In [9]:
# apply the function to the DataFrame
df['dish_id'] = df['URL'].apply(get_dish_id)
df.head()

,URL,AMOUNT,INGREDIENT,dish_id
0,https://www.bigoven.com/recipe/chicken-breasts...,2 tablespoons,canola oil,561575
1,https://www.bigoven.com/recipe/chicken-breasts...,1/2 large,onion,561575
2,https://www.bigoven.com/recipe/chicken-breasts...,6,pineapple,561575
3,https://www.bigoven.com/recipe/chicken-breasts...,,pepper,561575
4,https://www.bigoven.com/recipe/chicken-breasts...,1 1/2 cup,tomatillo,561575


### Create a column with dish name

In [10]:
# Write a function to get the dish name from URL
def get_dish_name(url): 
    dish_name = re.findall('\S*recipe/(\S*)/.*', url)[0]
    dish_name = re.sub('-', ' ', dish_name)
    return dish_name

In [11]:
df['dish_name'] = df['URL'].apply(get_dish_name)
df.head()

,URL,AMOUNT,INGREDIENT,dish_id,dish_name
0,https://www.bigoven.com/recipe/chicken-breasts...,2 tablespoons,canola oil,561575,chicken breasts with grilled pineapple and tom...
1,https://www.bigoven.com/recipe/chicken-breasts...,1/2 large,onion,561575,chicken breasts with grilled pineapple and tom...
2,https://www.bigoven.com/recipe/chicken-breasts...,6,pineapple,561575,chicken breasts with grilled pineapple and tom...
3,https://www.bigoven.com/recipe/chicken-breasts...,,pepper,561575,chicken breasts with grilled pineapple and tom...
4,https://www.bigoven.com/recipe/chicken-breasts...,1 1/2 cup,tomatillo,561575,chicken breasts with grilled pineapple and tom...


In [12]:
# Check top 5 dishes
df['dish_name'].value_counts()[:5]

korean bulgogi bbq beef tacos                                                48
grilled pork tenderloin                                                      43
grilled steak tacos with cilantro chimichurri sauce                          38
smokehouse pulled pork with memphis style barbecue sauce and classic slaw    34
grilled steak salad caesar style                                             34
Name: dish_name, dtype: int64

## Clean Columns

In [13]:
# since we got all information, remove URL column
df.drop('URL', axis = 1, inplace = True)
df.head()

,AMOUNT,INGREDIENT,dish_id,dish_name
0,2 tablespoons,canola oil,561575,chicken breasts with grilled pineapple and tom...
1,1/2 large,onion,561575,chicken breasts with grilled pineapple and tom...
2,6,pineapple,561575,chicken breasts with grilled pineapple and tom...
3,,pepper,561575,chicken breasts with grilled pineapple and tom...
4,1 1/2 cup,tomatillo,561575,chicken breasts with grilled pineapple and tom...


### Rearrange columns

In [14]:
# check all columns
df.columns

Index(['AMOUNT', 'INGREDIENT', 'dish_id', 'dish_name'], dtype='object')

In [15]:
# rearrange columns
df = df[['dish_id', 'dish_name', 'INGREDIENT', 'AMOUNT']]
df.head()

,dish_id,dish_name,INGREDIENT,AMOUNT
0,561575,chicken breasts with grilled pineapple and tom...,canola oil,2 tablespoons
1,561575,chicken breasts with grilled pineapple and tom...,onion,1/2 large
2,561575,chicken breasts with grilled pineapple and tom...,pineapple,6
3,561575,chicken breasts with grilled pineapple and tom...,pepper,
4,561575,chicken breasts with grilled pineapple and tom...,tomatillo,1 1/2 cup


## Inspect all records for 'grilled steak tacos with cilantro chimichurri sauce' to figure out the way to clean further

In [16]:
grilled_steak_tacos = df[df['dish_name'] == 'grilled steak tacos with cilantro chimichurri sauce']

In [17]:
grilled_steak_tacos

,dish_id,dish_name,INGREDIENT,AMOUNT
120202,1193837,grilled steak tacos with cilantro chimichurri ...,marinade,
120203,1193837,grilled steak tacos with cilantro chimichurri ...,steak,2 lbs
120204,1193837,grilled steak tacos with cilantro chimichurri ...,onion,1 sliced
120205,1193837,grilled steak tacos with cilantro chimichurri ...,orange,
120206,1193837,grilled steak tacos with cilantro chimichurri ...,lime,
120207,1193837,grilled steak tacos with cilantro chimichurri ...,soy sauce,1/3
120208,1193837,grilled steak tacos with cilantro chimichurri ...,olive oil,1/3
120209,1193837,grilled steak tacos with cilantro chimichurri ...,sugar,1/2 tsp
120210,1193837,grilled steak tacos with cilantro chimichurri ...,garlic,"4 smashed,"
120211,1193837,grilled steak tacos with cilantro chimichurri ...,cilantro,1/2 c


## Look at 'amt' Column

### What kind of ingredients are missing amounts? 

In [18]:
# How many?
missing_amt = df[df['AMOUNT'] == '']
missing_amt.shape

(1287, 4)

In [19]:
# Check a few example
missing_amt.head()

,dish_id,dish_name,INGREDIENT,AMOUNT
3,561575,chicken breasts with grilled pineapple and tom...,pepper,
31,860272,grilled chicken with pineapple salsa,olive oil,
32,860272,grilled chicken with pineapple salsa,pepper,
33,860272,grilled chicken with pineapple salsa,salt,
39,860272,grilled chicken with pineapple salsa,cilantro,


In [20]:
# Check the ingredients
missing_amt['INGREDIENT'].value_counts()

pepper                  105
black pepper             68
salt                     63
olive oil                45
garlic                   44
lemon                    43
kosher salt              38
cooking spray            31
chicken                  26
onion                    26
lime                     25
cilantro                 24
ground pepper            23
vegetable oil            22
chicken breast           18
tomatoes                 14
sugar                    14
brown sugar              13
cayenne                  13
spice                    13
mustard                  12
bbq                      12
paprika                  11
honey                    11
worcestershire sauce     11
marinade                 11
sesame seeds             11
parsley                  11
pork                     11
butter                   10
                       ... 
ricotta                   1
parmesan                  1
tahini                    1
celery seed               1
radicchio           

### Divide it by amount and unit

In [21]:
# How many different values? 
df['AMOUNT'].value_counts().shape

(1289,)

In [22]:
# First strip whitespace
df['AMOUNT'] = df['AMOUNT'].apply(lambda x: x.strip())

In [23]:
# How many different values? 
df['AMOUNT'].value_counts().shape

(1289,)

In [24]:
# Check top 20 values
df['AMOUNT'].value_counts()[:20]

                 1287
1 tablespoon      636
1/4 cup           535
2 tablespoons     497
1 teaspoon        484
1/2 cup           424
1/2 teaspoon      399
1                 370
1 cup             292
2                 248
1/4 teaspoon      231
2 teaspoons       211
3 tablespoons     174
4                 159
1/3 cup           129
2 tb              121
1 tb              108
1 tsp             103
2 cups            100
1 ts               93
Name: AMOUNT, dtype: int64

In [25]:
def get_amt(amt_str):
    """This function takes numeric amount from the amount string"""
    if len(amt_str) == 0:
        return ''
    elif len(re.findall('[^a-z]+', amt_str)) == 0: 
        return ''
    else: 
        return re.findall('[^a-z]+', amt_str)[0].strip()

In [26]:
df['amt_num'] = df['AMOUNT'].apply(get_amt)
df.head()

,dish_id,dish_name,INGREDIENT,AMOUNT,amt_num
0,561575,chicken breasts with grilled pineapple and tom...,canola oil,2 tablespoons,2
1,561575,chicken breasts with grilled pineapple and tom...,onion,1/2 large,1/2
2,561575,chicken breasts with grilled pineapple and tom...,pineapple,6,6
3,561575,chicken breasts with grilled pineapple and tom...,pepper,,
4,561575,chicken breasts with grilled pineapple and tom...,tomatillo,1 1/2 cup,1 1/2


In [27]:
def get_unit(amt_str):
    """This function takes unit from the amount string"""
    if len(amt_str) == 0:
        return ''
    elif len(re.findall('[A-Za-z]+', amt_str)) == 0: 
        return ''
    else: 
        return re.findall('[A-Za-z]+', amt_str)[0].strip()

In [28]:
df['unit'] = df['AMOUNT'].apply(get_unit)
df.head()

,dish_id,dish_name,INGREDIENT,AMOUNT,amt_num,unit
0,561575,chicken breasts with grilled pineapple and tom...,canola oil,2 tablespoons,2,tablespoons
1,561575,chicken breasts with grilled pineapple and tom...,onion,1/2 large,1/2,large
2,561575,chicken breasts with grilled pineapple and tom...,pineapple,6,6,
3,561575,chicken breasts with grilled pineapple and tom...,pepper,,,
4,561575,chicken breasts with grilled pineapple and tom...,tomatillo,1 1/2 cup,1 1/2,cup


In [29]:
# unit
print("Unit: {}".format(df['unit'].value_counts().shape))
print("\nValues:\n{}".format(df['unit'].value_counts()))

Unit: (276,)

Values:
               2479
cup            1596
teaspoon       1282
tablespoons     838
tablespoon      774
teaspoons       387
tb              324
tsp             308
ts              287
c               279
tbsp            275
cups            257
pound           179
large           174
pounds          166
cloves          163
lb              145
oz              134
medium          129
small            98
clove            79
ounces           69
ounce            66
whole            61
lbs              52
t                46
boneless         41
tbs              39
g                33
inch             31
               ... 
sweet             1
bag               1
carrots           1
ineapple          1
firm              1
eggplants         1
dashes            1
bulbs             1
star              1
leaf              1
anchovy           1
big               1
loaf              1
crushed           1
romaine           1
loaves            1
kilogram          1
finely            

In [30]:
# Numeric amount
print("Numeric amount: {}".format(df['amt_num'].value_counts().shape))
print("\nValues:\n{}".format(df['amt_num'].value_counts()))

Numeric amount: (294,)

Values:
1                3141
2                1997
                 1303
1/2              1264
1/4               979
4                 553
3                 549
1 1/2             304
1/3               177
3/4               175
6                 171
8                 131
5                  85
1/8                59
12                 45
16                 38
2/3                31
1 1/4              30
10                 29
2 1/2              24
1-1/2              22
1-2                20
0.5                13
1.5                13
2-3                13
100                12
3 1/2              12
3-4                10
20                  9
1 1/3               9
                 ... 
1 13-               1
1 14.5-             1
2 (8-               1
4 (10               1
2 (1/2              1
1 (15-              1
1 3 1/2             1
1 3                 1
80                  1
1         1/2       1
16 (                1
1 15.5-             1
6-7                 1
